In [ ]:
import sys

import tensorflow as tf
import tf_utils
from tensorflow_addons.metrics import CohenKappa
from tensorflow.keras.models import Model, load_model
from utils import *
from padding_generator import generator1
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Activation, GlobalAveragePooling2D, Dense, ZeroPadding2D, Dropout

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD, Nadam

import math

In [ ]:
MODEL_PATHS = data_paths('baseline')

input_shape = (224, 224)

input_img = Input(shape=(*input_shape, 1), name='input')

x = ZeroPadding2D(3)(input_img)
x = Conv2D(filters=64, kernel_size=7, strides=2, activation='relu')(x)
x = ZeroPadding2D(1)(x)
x = MaxPool2D(3, strides=2)(x)

x = Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')(x)
x = Dropout(0.3)(x)
x = Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')(x)
x = Dropout(0.3)(x)
x = MaxPool2D(2)(x)

x = Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(x)
x = Dropout(0.4)(x)
x = Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(x)
x = Dropout(0.4)(x)
x = MaxPool2D(2)(x)

x = Conv2D(filters=512, kernel_size=3, activation='relu', padding='same')(x)

x = GlobalAveragePooling2D(name='GAP')(x)
x = Dropout(0.5)(x)
x = Dense(1000, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[input_img], outputs=[output])

optimizer = Nadam(learning_rate=1e-4)

model.compile(
    optimizer=optimizer,
    metrics=[CohenKappa(num_classes=2), 'accuracy'],
    loss='binary_crossentropy',
)

In [ ]:
model.summary()

In [ ]:
set_note(MODEL_PATHS, 
        'Baseline model',
         summary_append=model
        )

model.save(MODEL_PATHS['init'])
print('Created: ' + MODEL_PATHS['init'])

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True)